In [ ]:
#Trexei alla arga, prolavame mono tis 3 prwtes
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import json
import ipywidgets as widgets
from IPython.display import display, Markdown
import logging

"""
Sentance Searcher


"""

logging.getLogger("pdfminer").setLevel(logging.ERROR)

def search_sentence_in_pdf(pdf_path, sentence):
    with pdfplumber.open(pdf_path) as pdf:
        for page_num in range(len(pdf.pages)):
            page = pdf.pages[page_num]
            text = page.extract_text()

            if text:
                # Normalize whitespace
                clean_text = ' '.join(text.split())

                # Split into sentences using punctuation
                sentence_list = re.split(r'(?<=[.!?])\s+', clean_text)

                # Find index of the sentence containing the search phrase
                for idx, sent in enumerate(sentence_list):
                    if sentence in sent:
                        # Get current + next 3 sentences
                        end_idx = min(idx + 11, len(sentence_list))
                        context_sentences = sentence_list[idx:end_idx]
                        return ' '.join(context_sentences)
    return " "

def search_in_multiple_pdfs(pdf_list, sentence):
    results = {}
    result = search_sentence_in_pdf(pdf_list, sentence)
    results[pdf_list] = result
    return results


pdf_files = ["pdf_2.pdf", "pdf_4.pdf", "pdf_5.pdf", "pdf_6.pdf", "pdf_7.pdf"]


# Load model
model = SentenceTransformer("multi-qa-MiniLM-L6-cos-v1")

# Load paragraph DB
paragraph_index = faiss.read_index("paragraph.index")
paragraph_texts = json.load(open("all_paragraphs.json"))
paragraph_sources = json.load(open("paragraph_sources.json"))

output = widgets.Output()

# Search function
def search_faiss(query, k=40):
    query_vec = model.encode([query]).astype("float32")
    distances, indices = paragraph_index.search(query_vec, k)
    results = [(paragraph_texts[i], paragraph_sources[i], distances[0][rank]) for rank, i in enumerate(indices[0])]

    return results


model_id = "microsoft/bitnet-b1.58-2B-4T"

def ask_bitnet(query):
# Load tokenizer and model
  tokenizer = AutoTokenizer.from_pretrained(model_id)
  model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16
  )

  results = search_faiss(query)
  top_paragraphs = "\n\n".join([text for text, _, _ in results[:5]])

# Apply the chat template
  messages = [
      {"role": "system", "content": f"""You are a helpful AI assistant. Use only the provided context to answer the question clearly and concisely.
      <<<
      context: {top_paragraphs}
      >>>

      <<<
      question: {query}
      >>>
      """},
      {"role": "user", "content": f"{query}"},
  ]
  prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
  chat_input = tokenizer(prompt, return_tensors="pt").to(model.device)

# Generate response
  chat_outputs = model.generate(**chat_input, max_new_tokens=50)
  response = tokenizer.decode(chat_outputs[0][chat_input['input_ids'].shape[-1]:], skip_special_tokens=True) # Decode only the response part
  return response

queries = [
    "How did Baker Hughes engineers use MATLAB to develop pump health monitoring software?",
    "Why is it important for training data in predictive maintenance systems to include instances from both normal and fault conditions?",
    "What is the recall performance of the proposed ENBANN method in comparison to other methods?",
    "What is cross-sectional prediction and how can it be applied in estimating component lifespan?",
    "Why are gas leak detectors important in environments with many pneumatic valves, and what type of detectors are considered non-intrusive?",
    "What new Industry 4.0 technologies are being used for remote asset monitoring, and what tools support them?",
    "What does the simulation model of the SUDM policy evaluate, and what assumptions are made about workstation operations?",
    "How were the prior parameters for the Weibull and exponential degradation models estimated, and what assumptions were made about the error terms?",
    "How does fuzzy logic contribute to diagnostics in machine failure and maintenance management?",
    "Why are artificial neural networks suitable for prognostics in machine failure, and what limitations do traditional systems face?",
    "How do Big Data platforms and CMMS contribute to the formulation of maintenance strategies?",
    "What is the relationship between diagnostics and prognostics in the context of machine degradation and failure?"
]

# Loop through each query and display results
for query_2 in queries:
    print(ask_bitnet(query_2))